In [ ]:
import pandas as pd
import os
import numpy as np
import seaborn as sns

def parse_table(i):
    new = [int(x) for x in i.replace("[", "").replace("]", "").split(",")]
    return [[new[0], new[1]], [new[2], new[3]]]

from scipy.stats import norm
from scipy.stats import fisher_exact
# Example p-value
p_value = 0.528643849635648

# Convert p-value to z-score
z_score = norm.ppf(1 - p_value / 2)
z_score

In [ ]:
#Setting up for TFBS
for cutt in [400, 500]:
    for cuttt in [0.5, 0.6, 0.7, 0.8, 0.9]:
        cols = []
        ind = 1
        #df = pd.read_csv("PerGene_Unfold_Permute/Finalizing_PerGene_FisherExact_NonCod_PhyloP447_MAFCut0.25_SpecSup250_PhyloPCut-100_PhastConsCut-1.csv").set_index("Gene")[["Permuted z-score"]]
        df = 0
        for f in os.listdir("TFBS_Results"):
            if "." not in f:
                for file in os.listdir("TFBS_Results/" + f):
                    if "AbsLogfc_HrefDneCrefOnly_All.txt" in file and "SpecSup0" in file and "FilterNewTEs" in file:
                        print(file)
                        v = pd.read_csv("TFBS_Results/" + f + "/" + file, sep = "\t")

                        v = v[v["Proportion"] == cuttt]
                        v = v[v["Cutoff"] == cutt]
                        v.index = v["TF_JASPAR_ID"] + "_" + v["TF_Name"]
                        v = v[["[[dc1, du1], [pc1, pu1]]", "Fisher exact p-value"]]
                        out = []
                        for index, row in v.iterrows():
                            p = fisher_exact(parse_table(row["[[dc1, du1], [pc1, pu1]]"]))
                            if p[0] < 1:
                                sign = -1
                            else:
                                sign = 1
                            out.append(sign*norm.ppf(1 - row["Fisher exact p-value"] / 2))
                        v["X"] = out
                        v = v[["X"]]
                        v.columns = [f]
                        if ind:
                            df = v
                            ind = 0
                        else:
                            df = df.join(v)
        #Replace inf values with large Z-score
        df = df.dropna()
        df = df.replace(-np.inf, -9)
        df = df.replace(np.inf, 9)
        
        df2 = df.copy()
        df.to_csv("Input_FilterNewTEs_ZScore_TFBS_SYNGO_MASH_Cutoff" + str(cutt) + "Proportion" + str(cuttt) + ".txt", sep = "\t")
        for i in df.columns:
            df[i] = np.repeat(1, df.shape[0])
        df.to_csv("Input_FilterNewTEs_SHat_TFBS_SYNGO_MASH_Cutoff" + str(cutt) + "Proportion" + str(cuttt) + ".txt", sep = "\t")

In [ ]:
#Setting up for PerGene

for cuttt in [0.5, 0.6, 0.7, 0.8, 0.9]:
    cols = []
    ind = 1
    #df = pd.read_csv("PerGene_Unfold_Permute/Finalizing_PerGene_FisherExact_NonCod_PhyloP447_MAFCut0.25_SpecSup250_PhyloPCut-100_PhastConsCut-1.csv").set_index("Gene")[["Permuted z-score"]]
    df = 0
    for f in os.listdir("PerGene_Unfold_Permute"):
        if "." not in f:
            for file in os.listdir("PerGene_Unfold_Permute/" + f):
                if "abs_logfc" in file and "Top20th" not in file and ".csv" in file and "PhastConsCut0" not in file and "FilterNewTEs" in file:
                    print(file)
                    v = pd.read_csv("PerGene_Unfold_Permute/" + f + "/" + file, sep = ",")

                    v = v[v["Proportion"] == cuttt]
                    v.index = v["Gene"]
                    v = v[["[[dc1, du1], [pc1, pu1]]", "Fisher exact p-value"]]
                    out = []
                    for index, row in v.iterrows():
                        p = fisher_exact(parse_table(row["[[dc1, du1], [pc1, pu1]]"]))
                        if p[0] < 1:
                            sign = -1
                        else:
                            sign = 1
                        out.append(sign*norm.ppf(1 - row["Fisher exact p-value"] / 2))
                    v["X"] = out
                    v = v[["X"]]
                    v.columns = [f]
                    if ind:
                        df = v
                        ind = 0
                    else:
                        df = df.join(v)
    #Replace inf       
    df = df.dropna()
    df = df.replace(np.inf, 9)
    df = df.replace(-np.inf, 9)
    df2 = df.copy()
    df.to_csv("Input_ZScore_PerGeneFilterNewTEs_MASH_Proportion" + str(cuttt) + ".txt", sep = "\t")
    for i in df.columns:
        df[i] = np.repeat(1, df.shape[0])
    df.to_csv("Input_SHat_PerGeneFilterNewTEs_MASH_Proportion" + str(cuttt) + ".txt", sep = "\t")

Finalizing_PerGene_FilterNewTEs_FisherExact_NonCod_DlLiangSteinNeuron_abs_logfc_All_MAFCut0.25_SpecSup0_PhyloPCut-100_PhastConsCut-1.csv.gz
Finalizing_PerGene_FilterNewTEs_FisherExact_NonCod_DlLiangSteinProgenitor_abs_logfc_All_MAFCut0.25_SpecSup0_PhyloPCut-100_PhastConsCut-1.csv.gz
Finalizing_PerGene_FilterNewTEs_FisherExact_NonCod_DlNeuralFemale_abs_logfc_All_MAFCut0.25_SpecSup0_PhyloPCut-100_PhastConsCut-1.csv.gz
Finalizing_PerGene_FilterNewTEs_FisherExact_NonCod_DlOGC_abs_logfc_All_MAFCut0.25_SpecSup0_PhyloPCut-100_PhastConsCut-1.csv.gz
Finalizing_PerGene_FilterNewTEs_FisherExact_NonCod_DlpreGC_IIaFemale_abs_logfc_All_MAFCut0.25_SpecSup0_PhyloPCut-100_PhastConsCut-1.csv.gz
Finalizing_PerGene_FilterNewTEs_FisherExact_NonCod_DlPVALB_abs_logfc_All_MAFCut0.25_SpecSup0_PhyloPCut-100_PhastConsCut-1.csv.gz
Finalizing_PerGene_FilterNewTEs_FisherExact_NonCod_DlSertoliMale_abs_logfc_All_MAFCut0.25_SpecSup0_PhyloPCut-100_PhastConsCut-1.csv.gz
Finalizing_PerGene_FilterNewTEs_FisherExact_NonCod

Finalizing_PerGene_FilterNewTEs_FisherExact_NonCod_DlFetalBrainRoofPlate_abs_logfc_All_MAFCut0.25_SpecSup0_PhyloPCut-100_PhastConsCut-1.csv.gz
Finalizing_PerGene_FilterNewTEs_FisherExact_NonCod_DlFetalChondrocytes_abs_logfc_All_MAFCut0.25_SpecSup0_PhyloPCut-100_PhastConsCut-1.csv.gz
Finalizing_PerGene_FilterNewTEs_FisherExact_NonCod_DlFetalGonadImmuneFemale_abs_logfc_All_MAFCut0.25_SpecSup0_PhyloPCut-100_PhastConsCut-1.csv.gz
Finalizing_PerGene_FilterNewTEs_FisherExact_NonCod_DlFetalHeartCardiacFibroblasts_abs_logfc_All_MAFCut0.25_SpecSup0_PhyloPCut-100_PhastConsCut-1.csv.gz
Finalizing_PerGene_FilterNewTEs_FisherExact_NonCod_DlFetalHeartEndocardium_abs_logfc_All_MAFCut0.25_SpecSup0_PhyloPCut-100_PhastConsCut-1.csv.gz
Finalizing_PerGene_FilterNewTEs_FisherExact_NonCod_DlFetalHeartPericytes_abs_logfc_All_MAFCut0.25_SpecSup0_PhyloPCut-100_PhastConsCut-1.csv.gz
Finalizing_PerGene_FilterNewTEs_FisherExact_NonCod_DlFetalLeydigMale_abs_logfc_All_MAFCut0.25_SpecSup0_PhyloPCut-100_PhastConsCut-